# Starbucks Capstone Challenge

***

## Table of Contents
* [Project Set-up by Udacity](#project_setup)

***

## Project Set-up by Udacity<a class="anchor" id="project_setup"></a>

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

### Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

***

## Data Cleaning

### Exploring the Data

In the following cell, I load the data from json files.

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio.join(pd.get_dummies(portfolio['channels'].apply(pd.Series).stack()).sum(level=0))

,channels,difficulty,duration,id,offer_type,reward,email,mobile,social,web
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,1,1,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,1,1,1
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0,1,1,0,1
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1,1,0,1
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,1,0,0,1
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3,1,1,1,1
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2,1,1,1,1
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0,1,1,1,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,1,1,1,1
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2,1,1,0,1


After checking the head lines of these three dataframes, I notice that if the `transcript` dataframe is sorted first by `person` and then by `time`, then I can trace the events happening to all customers over time. I am most interested in what happens after an offer is received for each (`person`, `offer id`) pair. Observing the data set, I find that after `offer received`, three types of event could happen over time, being `offer viewed`, `transaction`, and `offer completed`. I observe something counter-intuitive: for some (`person`, `offer id`) pairs, `offer completed` happens before `offer viewed`. I think this is because of some errors in data collection. Because of this observation, I decide to focus only on whether an offer is completed or not, and ignore whether an offer is viewed or not. Again, this is due to the errors in the dataset, and I would love to explore the latter question if a better dataset is available.

In [3]:
transcript.sort_values(by=['person', 'time'], ascending=True, inplace=True)
transcript = transcript.reset_index(drop=True)
transcript.head(20)

,event,person,time,value
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16}
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
5,offer received,0009655768c64bdeb2e877511632db8f,408,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
6,transaction,0009655768c64bdeb2e877511632db8f,414,{'amount': 8.57}
7,offer completed,0009655768c64bdeb2e877511632db8f,414,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...
8,offer viewed,0009655768c64bdeb2e877511632db8f,456,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
9,offer received,0009655768c64bdeb2e877511632db8f,504,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}


In [4]:
sum(transcript['event'] == 'offer received')

76277

### Transforming the data set

My goal is to understand what types of offer are more likely to be completed by customers in different demographic groups. To accomplish this goal, I need to create a dataframe, collecting information about whether the received offers are completed by the customers. 

I want to point out two tricky parts in the following code.
- The dictionary keys for the offer id in column `value` are `offer_id` if the corresponding event is `offer received`, while those are `offer id` for all other events. I made use of this while coding.
- It is possible that the same person receives offer A twice, and completes offer A for 0, 1, or 2 times. When there are multiple receiptions and completions of offers for the same (person, offer_id) pair, I assume the earliest received offer is completed also earlier. I realize this assumption by recording the first completed offer in `completed_index` and then only searching for the next completed offer if the same offer id is received for multiple times. Note that I also follow this assumption when collecting the time when offer is received or completed in `time_received` resp. `time_completed`.

In [ ]:
customer = [] # empty list to store the customer ids
offer_completed = [0] * transcript['event'].value_counts()['offer received'] # 1 if an offer is completed, 0 otherwise
completed_index = [] # a list to store indices of recorded completed offers to avoid counting the same completed offers for multiple times
offer_id = [] # to store the offer id
time_received = [] # to store the time when offer is received
time_completed = [1000] * transcript['event'].value_counts()['offer received'] # if not completed, value is 1000
index = -1 # to trace the number of the (person, offer id) pair

for i in np.arange(transcript.shape[0]):
    if transcript.iloc[i]['event'] == 'offer received':
        index += 1
        customer.append(transcript.iloc[i]['person'])
        offer_id.append(transcript.iloc[i]['value']['offer id'])
        time_received.append(transcript.iloc[i]['time'])
        j = i+1 # from the next index on, until the customer changes, check whether the particular offer is completed
        while j < transcript.shape[0] and transcript.iloc[j]['person'] == transcript.iloc[i]['person']:
            if transcript.iloc[j]['event'] == 'offer completed' and \
            transcript.iloc[j]['value']['offer_id'] == transcript.iloc[i]['value']['offer id'] and \
            j not in completed_index:
                offer_completed[index] = 1
                time_completed[index] = transcript.iloc[j]['time']
                completed_index.append(j)
                print(i, index, j, transcript.iloc[j]['event'], transcript.iloc[j]['time'])
                break 
            j += 1

In [11]:
df = pd.DataFrame(list(zip(customer, offer_id, offer_completed, time_received, time_completed)), \
                  columns = ['customer', 'offer_id', 'offer_completed', 'time_received', 'time_completed'])
# check that indeed I have counted the correct numbers of `offer received` and `offer completed`
print('The number of offer received in dataset transcript is ', sum(transcript['event'] == 'offer received'),'\n'\
      'The number of offer received in my df is ', df.shape[0], '\n'\
      'The number of offer completed in dataset transcript is ', sum(transcript['event'] == 'offer completed'),'\n'\
      'The number of offer completed in my df is ', df['offer_completed'].sum(),'\n'\
      'So I have collected the information into df correctly! ')
df.head()

The number of offer received in dataset transcript is  76277 
The number of offer received in my df is  76277 
The number of offer completed in dataset transcript is  33579 
The number of offer completed in my df is  33579 
So I have collected the information into df correctly! 


,customer,offer_id,offer_completed,time_received,time_completed
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,0,168,1000
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,0,336,1000
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,1,414,1000
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1,528,1000
4,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,1,576,1000


In [13]:
df.describe()

,offer_completed,time_received,time_completed
count,76277.000000,76277.000000,76277.0
mean,0.440224,365.124585,1000.0
std,0.496417,198.216556,0.0
min,0.000000,0.000000,1000.0
25%,0.000000,174.000000,1000.0
50%,0.000000,408.000000,1000.0
75%,1.000000,522.000000,1000.0
max,1.000000,714.000000,1000.0


In [ ]:
# as the cell to collect df takes long, 

In [12]:
# I use this cell to debug when I couldn't get the number of completed offer correctly. 
# This is how I found the tricky parts, which are noted in the beginning of this section.
tmp = transcript[transcript['event'] == 'offer completed']['person'].value_counts(ascending=True).to_frame().reset_index()
tmp2 = df[df['offer_completed'] == 1]['customer'].value_counts(ascending = True).to_frame().reset_index()
tmp3 = tmp.merge(tmp2)
# store the customers whose number of offer completed in transcript does not match with that in my df
customers = tmp3[tmp3['person'] != tmp3['customer']]['index'].tolist()
# 'person' contains number of offer received in transcript, customer is that number in my df
# if the output is empty, I am doing a good job!
tmp3[tmp3['person'] != tmp3['customer']].sort_values(by=['index'])

,index,person,customer


Now I have a nice dataframe storing all the offers received by customers, whether they are completed or not, and the time when these happened. I am ready to combine the information from `portfolio` and `profile`.